In [1]:
import logging
import numpy as np
import pandas as pd
import h5py as h5
from collections import namedtuple

In [2]:
logging.basicConfig(level=1, format='[%(asctime)s %(levelname)s] %(message)s')
Option = namedtuple("MyStruct", "input test_size")
                    # model name folds batch_size epochs")
                    #structure learning_rate regularizer epochs")
args = Option(
    input='data/total.h5',
    test_size = 0.75
    #model='share/reference_number.py',
    #name='NumberNetworkKFOLD',
    #folds=10,
    #batch_size=60,
    #epochs=20,
    #learning_rate=0.0001,
    #regularizer=0.0001,
)

In [3]:
logging.info(f'Loading data from {args.input}')

with h5.File(args.input, 'r') as data:
    data_x = data['input'][()]
    data_y = data['target'][()]



[2020-07-09 02:48:35,302 INFO] Loading data from data/total.h5


In [4]:
logging.info(f'Shuffling {args.input}')
data = np.concatenate([data_x, data_y], axis=1)
rng = np.random.default_rng(seed=42)
rng.shuffle(data)

data_x = data[:, 0:data_x.shape[1]]
data_y = data[:, data_x.shape[1]:data.shape[1]]

[2020-07-09 02:48:53,024 INFO] Shuffling data/total.h5


In [12]:
data_x.shape[0]

16248284

In [1]:
train_split = range(int(data_x.shape[0]*args.test_size))
train_split

NameError: name 'data_x' is not defined

In [26]:
test_split = range(int(data_x.shape[0]*args.test_size), data_x.shape[0])
test_split

range(14623455, 16248284)

In [23]:
data_x[train_split].shape

(14623455, 60)

In [24]:
data_x[test_split].shape

(1624829, 60)

In [27]:
with h5.File('data/train.h5', 'w') as hfile:
    hfile.create_dataset('input', data=data_x[train_split])
    hfile.create_dataset('target', data=data_y[train_split])

with h5.File('data/test.h5', 'w') as hfile:
    hfile.create_dataset('input', data=data_x[test_split])
    hfile.create_dataset('target', data=data_y[test_split])